# Train Anamoly Model

## Initialize Parameters from Data-processing notebook

In [14]:
!pip install -r requirements.txt > /dev/null 2>&1

In [18]:
# !pip install numpy
# !pip install tensorflow
# !pip install tensorflow-io
# !pip install tf-models-official

In [1]:
import os

In [2]:
bucket = os.getenv("BUCKET_NAME", "ml-data")

In [3]:
# os.environ["AWS_ACCESS_KEY_ID"] = "console"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "console123"
# os.environ["AWS_REGION"] = "us-east-1"
# os.environ["S3_ENDPOINT"] = "https://minio.svc.cluster.bla"
# os.environ["S3_USE_HTTPS"] = "1"
# os.environ["S3_VERIFY_SSL"] = "1"

In [4]:
import numpy as np
import tensorflow as tf
import tensorflow_io
from tensorflow.keras.utils import Sequence
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from official.nlp import optimization
from sklearn.utils import shuffle
from datetime import datetime

In [5]:
np_data = np.load('processed_train_data.npz', allow_pickle=True)
x_tr = np_data['x_train']
y_tr = np_data['y_train']
embed_dim = 768
max_len = 75
fine_tune_model_remote_path = None
batch_size = 64
epochs = 10
model_name = "bert_hdfs_weights.hdf5"

## Helper Functions for Model Training

### Create positional embedding layer (Custom Layer)

In [6]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(PositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_encoding = positional_encoding(max_len,
                                                embed_dim)
 
    def call(self, x):
        seq_len = tf.shape(x)[1]
        x += self.pos_encoding[:, :seq_len, :]
        return x

### Batch generator class 

In [7]:
class BatchGenerator(Sequence):

    def __init__(self, X, Y, batch_size):
        self.X, self.Y = X, Y
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.X) / float(self.batch_size)))

    def __getitem__(self, idx):
        # print(self.batch_size)
        dummy = np.zeros(shape=(embed_dim,))
        x = self.X[idx * self.batch_size:min((idx + 1) * self.batch_size, len(self.X))]
        X = np.zeros((len(x), max_len, embed_dim))
        Y = np.zeros((len(x), 2))
        item_count = 0
        for i in range(idx * self.batch_size, min((idx + 1) * self.batch_size, len(self.X))):
            x = self.X[i]
            if len(x) > max_len:
                x = x[-max_len:]
            x = np.pad(np.array(x), pad_width=((max_len - len(x), 0), (0, 0)), mode='constant',
                       constant_values=0)
            X[item_count] = np.reshape(x, [max_len, embed_dim])
            Y[item_count] = self.Y[i]
            item_count += 1
        return X[:], Y[:, 0]

### Anomaly Model

In [8]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim), ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
    
def transformer_classifer(embed_dim, ff_dim, max_len, num_heads, dropout=0.1):
    inputs = layers.Input(shape=(max_len, embed_dim))
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    embedding_layer = PositionEmbedding(100, 2000, embed_dim)
    x = embedding_layer(inputs)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(32, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(2, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

## Helper functions for training the model

In [9]:
def train_generator(training_generator, validate_generator, num_train_samples, num_val_samples, batch_size,
                    epoch_num, model_name=None):
    epochs = epoch_num
    steps_per_epoch = num_train_samples
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1 * num_train_steps)

    init_lr = 3e-4
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                              num_train_steps=num_train_steps,
                                              num_warmup_steps=num_warmup_steps,
                                              optimizer_type='adamw')

    loss_object = SparseCategoricalCrossentropy()

    model = transformer_classifer(768, ff_dim=2048, max_len=75, num_heads=12, dropout=0.1)

    # model.load_weights("hdfs_transformer.hdf5")

    model.compile(loss=loss_object, metrics=['accuracy'],
                  optimizer=optimizer)

    print(model.summary())

    # checkpoint
    filepath = model_name
    checkpoint = ModelCheckpoint(filepath,
                                 monitor='val_accuracy',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='max',
                                 save_weights_only=True)
    early_stop = EarlyStopping(
        monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto',
        baseline=None, restore_best_weights=True
    )

    logdir = f"s3://{bucket}/logs/hdfs-tb/run-" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
    callbacks_list = [checkpoint, tensorboard_callback, early_stop]

    history = model.fit(training_generator,
                        steps_per_epoch=int(num_train_samples / batch_size),
                        epochs=epoch_num,
                        verbose=1,
                        validation_data=validate_generator,
                        validation_steps=int(num_val_samples / batch_size),
                        workers=16,
                        max_queue_size=32,
                        callbacks=callbacks_list,
                        shuffle=True
                        )
    return model, history


def train(X, Y, epoch_num, batch_size, model_file=None):
    X, Y = shuffle(X, Y)
    n_samples = len(X)
    train_x, train_y = X[:int(n_samples * 90 / 100)], Y[:int(n_samples * 90 / 100)]
    val_x, val_y = X[int(n_samples * 90 / 100):], Y[int(n_samples * 90 / 100):]

    training_generator, num_train_samples = BatchGenerator(train_x, train_y, batch_size), len(train_x)
    validate_generator, num_val_samples = BatchGenerator(val_x, val_y, batch_size), len(val_x)

    print("Number of training samples: {0} - Number of validating samples: {1}".format(num_train_samples,
                                                                                       num_val_samples))

    model, history = train_generator(
        training_generator,
        validate_generator,
        num_train_samples,
        num_val_samples,
        batch_size,
        epoch_num,
        model_name=model_file)

    return model, history

## Train the Model

In [10]:
model, history = train(x_tr, y_tr, epochs, batch_size, model_name)

Number of training samples: 280 - Number of validating samples: 32
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 75, 768)]         0         
                                                                 
 position_embedding (Positio  (None, 75, 768)          0         
 nEmbedding)                                                     
                                                                 
 transformer_block (Transfor  (None, 75, 768)          31491584  
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 768)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_2 (Dropout)         (None, 768)               0    

Layer PositionEmbedding has arguments ['max_len', 'vocab_size', 'embed_dim']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Epoch 1/20
4/4 [==============================] - ETA: 0s - loss: 0.2079 - accuracy: 0.9844WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


4/4 [==============================] - 21s 5s/step - loss: 0.2079 - accuracy: 0.9844
Epoch 2/20
4/4 [==============================] - ETA: 0s - loss: 0.1450 - accuracy: 0.9844WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


4/4 [==============================] - 19s 5s/step - loss: 0.1450 - accuracy: 0.9844
Epoch 3/20
4/4 [==============================] - ETA: 0s - loss: 0.1236 - accuracy: 0.9769WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


...

4/4 [==============================] - 16s 4s/step - loss: 0.1038 - accuracy: 0.9769
Epoch 17/20
4/4 [==============================] - ETA: 0s - loss: 0.0625 - accuracy: 0.9805WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


4/4 [==============================] - 19s 5s/step - loss: 0.0625 - accuracy: 0.9805
Epoch 18/20
4/4 [==============================] - ETA: 0s - loss: 0.1239 - accuracy: 0.9769WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


4/4 [==============================] - 16s 5s/step - loss: 0.1239 - accuracy: 0.9769
Epoch 19/20
4/4 [==============================] - ETA: 0s - loss: 0.0908 - accuracy: 0.9815WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


4/4 [==============================] - 16s 4s/step - loss: 0.0908 - accuracy: 0.9815
Epoch 20/20
4/4 [==============================] - ETA: 0s - loss: 0.1496 - accuracy: 0.9769WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


4/4 [==============================] - 16s 4s/step - loss: 0.1496 - accuracy: 0.9769


In [18]:
model.save_weights(model_name)

In [11]:
print("="*100)
print("")
print("DONE TRAINING")
print(os.getcwd())
print("")
print("="*100)


DONE TRAINING
/home/jovyan/minio-ml/log-trainer



In [12]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    print(f)

processed_test_data.npz
minio-train-model.ipynb
sample-secret.yaml
NeuralLog-trainer.pipeline
processed_train_data.npz
data-bert.npz
minio-evaluate-model.ipynb
requirements.txt
minio-logs-data-processing.ipynb
mlpipeline-metrics.json


In [13]:
metrics = {'metrics': []}
for key, value in history.history.items():
    metrics['metrics'].append({
        'name': key,
        'numberValue': value[0]
    })

In [14]:
import json
with open('mlpipeline-metrics.json', 'w') as f:
    json.dump(metrics, f)